#🎮Trabajo Final -  Consultas Inteligentes sobre Equipos del FIFA 23
Este trabajo final integra técnicas de procesamiento de lenguaje natural con modelos de embeddings y un LLM para permitir consultas en lenguaje natural sobre datos de equipos del **FIFA 23**.

🧾 Los datos fueron obtenidos de un archivo CSV con estadísticas detalladas de clubes como:
- Nombre del equipo
- Liga y país
- Estadísticas ofensivas y defensivas (ataque, mediocampo, defensa)

⚙️ La app utiliza:
- **HuggingFace Embeddings** para vectorizar los datos (GPU si está disponible)
- **Gemini (Google)** como modelo de lenguaje para responder consultas ( Modelo: gemini-2.0-flash)
- **LlamaIndex** para realizar búsquedas semánticas sobre los datos
- **Streamlit** para la interfaz interactiva

##Instalacion de dependencias

In [1]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-langchain langchain-community langchain-google-genai google-generativeai pandas torch --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


##Lectura y limpieza del dataset

In [3]:
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive')
#ruta_csv = "/content/drive/MyDrive/Trabajo Final Inteligentes - Juan/equipos_filtrados.csv"

ruta_csv = "equipos_filtrados.csv"
df = pd.read_csv(ruta_csv)
df.head()

,id_equipo,nombre_equipo,nombre_liga,nivel_liga,nombre_nacionalidad,valoracion_general,ataque,mediocampo,defensa,estadio_local,...,tactica_corners,tactica_tiros_libres,velocidad_construccion,regate_construccion,pase_construccion,posicionamiento_construccion,pase_creacion_oportunidades,centros_creacion_oportunidades,remates_creacion_oportunidades,posicionamiento_creacion_oportunidades
0,10,Manchester City,Premier League,1.0,England,85,85,86,86,Etihad Stadium,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73,Paris Saint Germain,Ligue 1,1.0,France,85,87,83,83,Parc des Princes,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,243,Real Madrid,La Liga,1.0,Spain,85,85,86,84,Estadio Santiago Bernabéu,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1337,Germany,Friendly International,NaN,Germany,85,82,85,82,Stadion Olympik,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Chelsea,Premier League,1.0,England,84,83,83,83,Stamford Bridge,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Despliegue

In [4]:
!pip install streamlit pyngrok --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Creando .py para aplicacion Streamlit

In [ ]:
%%writefile equipos.py
import os
import pandas as pd
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_google_genai import ChatGoogleGenerativeAI
import torch
import streamlit as st
from llama_index.core import load_index_from_storage, VectorStoreIndex, Document, Settings
from llama_index.core.storage import StorageContext
import zipfile
import gdown

os.environ["HUGGINGFACEHUB_API_TOKEN"] = st.secrets["HUGGINGFACEHUB_API_TOKEN"]
os.environ["GOOGLE_API_KEY"] = st.secrets["GOOGLE_API_KEY"]
# os.environ["GOOGLE_API_KEY"] = "" #PONER API KEY DE GOOGLE

# Cargar los jugadores
#df = pd.read_csv("jugadores_filtrados.csv")
#df = pd.read_csv("jugadores_RealMadrid.csv")

# Convertir cada fila a un documento de texto para indexar
#documents = []
#for _, row in df.iterrows():
#    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
#    documents.append(Document(text=text))

# Usar embeddings con GPU si hay disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",#all-MiniLM-L12-v2/all-MiniLM-L6-v2/all-mpnet-base-v2
    device=device
)

# Cliente Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Crear contexto de servicio
Settings.llm = llm
Settings.embed_model = embed_model

# Creando indice
#index = VectorStoreIndex.from_documents(documents)
#query_engine = index.as_query_engine(similarity_top_k=3)

# # Guardar el indice
# from google.colab import drive
# drive.mount('/content/drive')
# persist_dir = "/content/drive/MyDrive/indice_jugadores"
# index.storage_context.persist(persist_dir=persist_dir)

file_id = "1j279vniYYGMop6lf7Me9cDc4NzBj5w_h"  #file id
zip_filename = "indice_equipos.zip"

if not os.path.exists("indice_equipos"):
    # Descargar desde Google Drive
    url = f"https://drive.google.com/uc?id={file_id}"
    output = gdown.download(url, zip_filename, quiet=False)

    # Descomprimir
    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        os.makedirs("indice_equipos", exist_ok=True)
        zip_ref.extractall("indice_equipos")
        

#Cargar carpeta con indices para que tarde menos
@st.cache_resource
def cargar_query_engine():
    # Cargar el storage context desde el directorio
    storage_context = StorageContext.from_defaults(persist_dir="indice_equipos")
    # Cargar el indice desde el contexto
    index = load_index_from_storage(storage_context)
    # Crear el query engine
    return index.as_query_engine(similarity_top_k=3)

query_engine = cargar_query_engine()
# Interfaz de usuario
st.set_page_config(
    page_title="Consultas FIFA 23 - Equipos",
    page_icon="⚽",
    layout="centered"
)

st.title("Consultas FIFA 23 - Equipos")
st.markdown("### 🧠 Ejemplos de preguntas que podes hacer:")

ejemplos = [
    "¿Cuanto ataque tiene Real Madrid?",
    "¿Cuanto mediocampo tiene FC Barcelona?",
    "¿En que liga juega Boca Juniors?",
    "¿Cuanta defensa tiene River Plate?"
]

# Mostrar los ejemplos con botones
for i, ejemplo in enumerate(ejemplos):
    col1, col2 = st.columns([0.8, 0.2])
    with col1:
        st.markdown(f"- {ejemplo}")
    with col2:
        if st.button("Consultar", key=f"btn_{i}"):
            st.session_state["consulta"] = ejemplo

# Campo de entrada manual
consulta = st.text_input("Ingresa tu consulta:", value=st.session_state.get("consulta", ""))

# Ejecutar si hay una consulta
if consulta:
    with st.spinner("⏳ Procesando tu consulta..."):
        consulta_modificada = consulta.strip() + ". Responde en español."
        respuesta_llm = llm.invoke(consulta_modificada)
        respuesta_rag = query_engine.query(consulta_modificada)

    st.success("✅ Consulta completada")

    st.markdown("### 💬 Respuesta sin RAG:")
    st.write(respuesta_llm.content)

    st.markdown("### 🔍 Respuesta con RAG:")
    st.write(str(respuesta_rag))
    print(consulta_modificada, str(respuesta_rag))

Overwriting equipos.py


## 🌐 Aplicacion publica con NGrok

In [ ]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = "" # PONER TOKEN NGROK
# Crear tunel
public_url = ngrok.connect(addr="8501", proto="http")
print("URL publica:\n", public_url)
# Ejecutar Streamlit
!streamlit run app.py &
# 1 minuto aprox en levantar (cargando indices base2)

URL publica:
 NgrokTunnel: "https://85a9-35-237-169-202.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.169.202:8501

2025-06-28 12:57:26.769182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751115446.794032    3300 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751115446.802168    3300 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-28 12:57:26.830812: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perform